In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import Input, Model, layers, losses, optimizers

from matplotlib import pyplot as plt

2022-12-02 13:36:00.101913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 13:36:00.164770: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def make_encoder(input_shape=(28,28,1), latent_dim=2):
    inx = Input(input_shape)
    x = layers.Conv2D(32, 3, strides=2, padding='same', activation='relu')(inx)
    x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(2*latent_dim)(x)
    return Model(inx, x)

enc = make_encoder()
enc.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 4)                 2052  

2022-12-02 13:36:01.614376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 13:36:02.049565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14349 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


In [3]:
def make_decoder(input_shape=(28,28,1), latent_dim=2):
    inx = Input(latent_dim)
    x = layers.Dense(512, activation='relu')(inx)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Reshape((4,4,128))(x)
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Cropping2D(cropping=(1, 1))(x)
    x = layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation=None)(x)
    return Model(inx, x)

dec = make_decoder()
dec.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 512)               1536      
                                                                 
 dense_3 (Dense)             (None, 2048)              1050624   
                                                                 
 reshape (Reshape)           (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 64)         73792     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 16, 16, 32)       18464     
 ranspose)                                                 

In [5]:
class VAE(Model):
    
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.latent_dim = self.decoder.input.shape[-1]
        
    def call(self, x):
        mean, logvar = self.encode(x)
        z = self.sample(mean, logvar)
        x_ = self.decoder(z)
        kl_loss = self.kl_analytic(mean, logvar)
        self.add_loss(tf.reduce_mean(kl_loss))
        self.add_metric(kl_loss, name='kl')
        return x_
    
    def kl_analytic(self, mean, logvar):
        kl = -.5*tf.reduce_mean(logvar - tf.square(mean) - tf.exp(logvar) + 1, axis=[1])
        return kl
        
    def sample(self, mean, logvar):
        batch_size = tf.shape(mean)[0]
        eps = tf.random.normal((batch_size, self.latent_dim))
        return eps*tf.exp(logvar*.5) + mean
    
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=-1)
        return mean, logvar

In [6]:
class MSELoss(losses.Loss):
    
    def call(x_true, x_rec):
        loss = tf.sum(tf.square(x_true-x_rec), axis=[1,2,3])
        return loss

In [7]:
loss = MSELoss()
opt = optimizers.Adam(learning_rate=1e-3)
vae = VAE(enc, dec)
vae.compile(loss=loss, optimizer=opt)

 - Load the MNIST dataset, rescale the values to lie [0,1] and split into train/test/rest sets (10000/1000/-1).
 - Train the VAE model, visualize the latent space and plot some inputs and their reconstruction.
 - Sample the latent space in a rectangluar grid and generate synthetic samples. Visualize them.
 - Binarize the MNIST dataset. What is the appropriate loss function for this data?

In [ ]:
# training

In [ ]:
# sampling

In [ ]:
# visualization

 - Train a classifier on the training data and on 10000 synthetic data points
 - Compare their performance on the test set
 Hint: Ensure that the synthetic data looks the same as the original data (values in [0,1])

 - Modify the VAE model to take a parameter beta that weights the KL-Term
 - Experiment with different values of beta. How does the reconstruction quality change? What about the latent embedding?

Is synthetic data generation sufficient to guarantee privacy? What do you think?